In [1]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
# load dataset
dataset = read_csv('Input_Dataset_LSTM.csv', header=0,)
values = dataset.values

In [27]:
dataset

,30.015715,0.6,0.005945025,0.422222222,0.4,0.006990737,-1
0,130.309998,0.6,0.005945,0.422222,0.4,0.018429,-1
1,298.612823,0.6,0.005945,0.422222,0.2,0.013547,-1
2,23.810173,0.6,0.005945,0.422222,0.6,0.004879,-1
3,129.479996,0.6,0.005945,0.422222,0.2,0.008508,-1
4,24.690001,0.6,0.005945,0.422222,0.4,0.006339,1
5,20.950001,0.6,0.005945,0.422222,0.4,0.006075,1
6,13.680000,0.6,0.005945,0.422222,0.8,0.017911,-1
7,30.270000,0.6,0.005945,0.422222,0.4,0.007295,-1
8,29.674286,0.4,0.008804,0.355556,0.2,0.008920,-1
9,127.349998,0.4,0.008804,0.355556,0.2,0.021792,-1


In [4]:
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)
# specify the number of lag hours
n_hours = 1
n_features = 6
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 0)

In [5]:
# split into train and test sets
values = reframed.values


In [6]:
n_epoch = 30
w = 0
r = 1
runs = [0]*30

for j in range(0,30):
	n_train_hours = 10000
	train = values[:n_train_hours, :]
	test = values[n_train_hours:, :]
	# split into input and outputs
	n_obs = n_hours * n_features
	train_X, train_y = train[:, :n_obs], train[:,-1]
	test_X, test_y = test[:, :n_obs], test[:, -1]
	# # reshape input to be 3D [samples, timesteps, features]
	train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
	test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
	# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
	#
	# # design network
	model = Sequential()
	model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
	model.add(Dense(1))
	model.compile(loss='mae', optimizer='adam')
	# fit network
	history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
	# plot history
	# pyplot.plot(history.history['loss'], label='train')
	# pyplot.plot(history.history['val_loss'], label='test')
	# pyplot.legend()
	# pyplot.show()





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10000 samples, validate on 5811 samples
Epoch 1/50
 - 3s - loss: 0.9318 - val_loss: 0.8208
Epoch 2/50
 - 1s - loss: 0.7967 - val_loss: 0.7763
Epoch 3/50
 - 2s - loss: 0.7806 - val_loss: 0.7650
Epoch 4/50
 - 1s - loss: 0.7700 - val_loss: 0.7568
Epoch 5/50
 - 0s - loss: 0.7629 - val_loss: 0.7533
Epoch 6/50
 - 1s - loss: 0.7606 - val_loss: 0.7529
Epoch 7/50
 - 0s - loss: 0.7599 - val_loss: 0.7523
Epoch 8/50
 - 1s - loss: 0.7592 - val_loss: 0.7512
Epoch 9/50
 - 1s - loss: 0.7581 - val_loss: 0.7499
Epoch 10/50
 - 1s - loss: 0.7573 - val_loss: 0.7487
Epoch 11/50
 - 1s - loss: 0.7560 - val_loss: 0.7475
Epoch 12/50
 - 1s - loss: 0.7554 - val_loss: 0.7464
Epoch 13/50
 - 1s - loss: 0.7544 - val_loss: 0.7447
Epoch 14/50
 - 1s - loss: 0.7530 - val_loss: 0.7437
Epoch 15/50
 - 1s - loss: 0.7520 - val_loss: 0.7425
Epoch 16/50
 - 1s - loss: 0.7509 - val_loss: 0.7412
Epoch 17/50
 - 1s - loss: 0.

Epoch 28/50
 - 0s - loss: 0.7241 - val_loss: 0.7141
Epoch 29/50
 - 0s - loss: 0.7210 - val_loss: 0.7105
Epoch 30/50
 - 0s - loss: 0.7175 - val_loss: 0.7070
Epoch 31/50
 - 0s - loss: 0.7148 - val_loss: 0.7034
Epoch 32/50
 - 0s - loss: 0.7115 - val_loss: 0.6999
Epoch 33/50
 - 0s - loss: 0.7082 - val_loss: 0.6969
Epoch 34/50
 - 0s - loss: 0.7051 - val_loss: 0.6932
Epoch 35/50
 - 0s - loss: 0.7020 - val_loss: 0.6903
Epoch 36/50
 - 0s - loss: 0.6997 - val_loss: 0.6873
Epoch 37/50
 - 0s - loss: 0.6975 - val_loss: 0.6854
Epoch 38/50
 - 0s - loss: 0.6962 - val_loss: 0.6840
Epoch 39/50
 - 0s - loss: 0.6948 - val_loss: 0.6823
Epoch 40/50
 - 0s - loss: 0.6935 - val_loss: 0.6809
Epoch 41/50
 - 0s - loss: 0.6921 - val_loss: 0.6795
Epoch 42/50
 - 0s - loss: 0.6910 - val_loss: 0.6782
Epoch 43/50
 - 0s - loss: 0.6899 - val_loss: 0.6767
Epoch 44/50
 - 0s - loss: 0.6887 - val_loss: 0.6753
Epoch 45/50
 - 0s - loss: 0.6875 - val_loss: 0.6739
Epoch 46/50
 - 0s - loss: 0.6864 - val_loss: 0.6726
Epoch 47/50


Epoch 34/50
 - 0s - loss: 0.7022 - val_loss: 0.6895
Epoch 35/50
 - 1s - loss: 0.6990 - val_loss: 0.6866
Epoch 36/50
 - 0s - loss: 0.6965 - val_loss: 0.6843
Epoch 37/50
 - 0s - loss: 0.6944 - val_loss: 0.6822
Epoch 38/50
 - 0s - loss: 0.6928 - val_loss: 0.6806
Epoch 39/50
 - 1s - loss: 0.6913 - val_loss: 0.6787
Epoch 40/50
 - 0s - loss: 0.6899 - val_loss: 0.6771
Epoch 41/50
 - 0s - loss: 0.6886 - val_loss: 0.6758
Epoch 42/50
 - 0s - loss: 0.6877 - val_loss: 0.6744
Epoch 43/50
 - 0s - loss: 0.6863 - val_loss: 0.6729
Epoch 44/50
 - 0s - loss: 0.6852 - val_loss: 0.6713
Epoch 45/50
 - 0s - loss: 0.6839 - val_loss: 0.6704
Epoch 46/50
 - 0s - loss: 0.6826 - val_loss: 0.6689
Epoch 47/50
 - 0s - loss: 0.6812 - val_loss: 0.6670
Epoch 48/50
 - 0s - loss: 0.6801 - val_loss: 0.6659
Epoch 49/50
 - 0s - loss: 0.6787 - val_loss: 0.6642
Epoch 50/50
 - 0s - loss: 0.6773 - val_loss: 0.6623
Train on 10000 samples, validate on 5811 samples
Epoch 1/50
 - 3s - loss: 0.9444 - val_loss: 0.8400
Epoch 2/50
 - 0s

Epoch 40/50
 - 1s - loss: 0.6904 - val_loss: 0.6771
Epoch 41/50
 - 0s - loss: 0.6890 - val_loss: 0.6756
Epoch 42/50
 - 0s - loss: 0.6878 - val_loss: 0.6740
Epoch 43/50
 - 1s - loss: 0.6864 - val_loss: 0.6722
Epoch 44/50
 - 0s - loss: 0.6851 - val_loss: 0.6706
Epoch 45/50
 - 0s - loss: 0.6838 - val_loss: 0.6691
Epoch 46/50
 - 0s - loss: 0.6824 - val_loss: 0.6674
Epoch 47/50
 - 0s - loss: 0.6811 - val_loss: 0.6656
Epoch 48/50
 - 0s - loss: 0.6797 - val_loss: 0.6641
Epoch 49/50
 - 0s - loss: 0.6782 - val_loss: 0.6621
Epoch 50/50
 - 0s - loss: 0.6768 - val_loss: 0.6602
Train on 10000 samples, validate on 5811 samples
Epoch 1/50
 - 3s - loss: 0.9218 - val_loss: 0.8113
Epoch 2/50
 - 0s - loss: 0.7900 - val_loss: 0.7708
Epoch 3/50
 - 1s - loss: 0.7754 - val_loss: 0.7603
Epoch 4/50
 - 1s - loss: 0.7659 - val_loss: 0.7545
Epoch 5/50
 - 0s - loss: 0.7614 - val_loss: 0.7534
Epoch 6/50
 - 0s - loss: 0.7600 - val_loss: 0.7528
Epoch 7/50
 - 0s - loss: 0.7593 - val_loss: 0.7516
Epoch 8/50
 - 0s - los

Epoch 46/50
 - 1s - loss: 0.6759 - val_loss: 0.6599
Epoch 47/50
 - 1s - loss: 0.6745 - val_loss: 0.6584
Epoch 48/50
 - 1s - loss: 0.6731 - val_loss: 0.6565
Epoch 49/50
 - 1s - loss: 0.6718 - val_loss: 0.6552
Epoch 50/50
 - 1s - loss: 0.6707 - val_loss: 0.6534
Train on 10000 samples, validate on 5811 samples
Epoch 1/50
 - 7s - loss: 0.9413 - val_loss: 0.8362
Epoch 2/50
 - 1s - loss: 0.7994 - val_loss: 0.7766
Epoch 3/50
 - 1s - loss: 0.7802 - val_loss: 0.7645
Epoch 4/50
 - 1s - loss: 0.7694 - val_loss: 0.7564
Epoch 5/50
 - 1s - loss: 0.7626 - val_loss: 0.7534
Epoch 6/50
 - 1s - loss: 0.7599 - val_loss: 0.7530
Epoch 7/50
 - 1s - loss: 0.7591 - val_loss: 0.7520
Epoch 8/50
 - 1s - loss: 0.7583 - val_loss: 0.7507
Epoch 9/50
 - 1s - loss: 0.7573 - val_loss: 0.7494
Epoch 10/50
 - 1s - loss: 0.7561 - val_loss: 0.7483
Epoch 11/50
 - 1s - loss: 0.7549 - val_loss: 0.7465
Epoch 12/50
 - 1s - loss: 0.7538 - val_loss: 0.7450
Epoch 13/50
 - 1s - loss: 0.7522 - val_loss: 0.7434
Epoch 14/50
 - 1s - loss

 - 4s - loss: 0.9278 - val_loss: 0.8200
Epoch 2/50
 - 0s - loss: 0.7907 - val_loss: 0.7699
Epoch 3/50
 - 0s - loss: 0.7730 - val_loss: 0.7580
Epoch 4/50
 - 0s - loss: 0.7638 - val_loss: 0.7539
Epoch 5/50
 - 1s - loss: 0.7608 - val_loss: 0.7537
Epoch 6/50
 - 0s - loss: 0.7599 - val_loss: 0.7531
Epoch 7/50
 - 0s - loss: 0.7591 - val_loss: 0.7518
Epoch 8/50
 - 0s - loss: 0.7583 - val_loss: 0.7508
Epoch 9/50
 - 0s - loss: 0.7573 - val_loss: 0.7496
Epoch 10/50
 - 0s - loss: 0.7564 - val_loss: 0.7484
Epoch 11/50
 - 0s - loss: 0.7554 - val_loss: 0.7469
Epoch 12/50
 - 0s - loss: 0.7543 - val_loss: 0.7455
Epoch 13/50
 - 0s - loss: 0.7530 - val_loss: 0.7441
Epoch 14/50
 - 0s - loss: 0.7517 - val_loss: 0.7427
Epoch 15/50
 - 0s - loss: 0.7502 - val_loss: 0.7410
Epoch 16/50
 - 0s - loss: 0.7488 - val_loss: 0.7398
Epoch 17/50
 - 1s - loss: 0.7476 - val_loss: 0.7381
Epoch 18/50
 - 0s - loss: 0.7459 - val_loss: 0.7370
Epoch 19/50
 - 0s - loss: 0.7442 - val_loss: 0.7348
Epoch 20/50
 - 0s - loss: 0.7423

Epoch 7/50
 - 1s - loss: 0.7599 - val_loss: 0.7517
Epoch 8/50
 - 1s - loss: 0.7593 - val_loss: 0.7511
Epoch 9/50
 - 1s - loss: 0.7585 - val_loss: 0.7501
Epoch 10/50
 - 1s - loss: 0.7576 - val_loss: 0.7489
Epoch 11/50
 - 1s - loss: 0.7568 - val_loss: 0.7477
Epoch 12/50
 - 1s - loss: 0.7557 - val_loss: 0.7463
Epoch 13/50
 - 1s - loss: 0.7549 - val_loss: 0.7452
Epoch 14/50
 - 1s - loss: 0.7540 - val_loss: 0.7444
Epoch 15/50
 - 1s - loss: 0.7532 - val_loss: 0.7433
Epoch 16/50
 - 1s - loss: 0.7523 - val_loss: 0.7422
Epoch 17/50
 - 1s - loss: 0.7511 - val_loss: 0.7409
Epoch 18/50
 - 1s - loss: 0.7496 - val_loss: 0.7401
Epoch 19/50
 - 1s - loss: 0.7484 - val_loss: 0.7387
Epoch 20/50
 - 1s - loss: 0.7472 - val_loss: 0.7378
Epoch 21/50
 - 1s - loss: 0.7457 - val_loss: 0.7360
Epoch 22/50
 - 1s - loss: 0.7442 - val_loss: 0.7344
Epoch 23/50
 - 1s - loss: 0.7424 - val_loss: 0.7328
Epoch 24/50
 - 1s - loss: 0.7403 - val_loss: 0.7305
Epoch 25/50
 - 1s - loss: 0.7380 - val_loss: 0.7281
Epoch 26/50
 - 

Epoch 13/50
 - 1s - loss: 0.7540 - val_loss: 0.7456
Epoch 14/50
 - 1s - loss: 0.7530 - val_loss: 0.7440
Epoch 15/50
 - 1s - loss: 0.7515 - val_loss: 0.7423
Epoch 16/50
 - 1s - loss: 0.7505 - val_loss: 0.7411
Epoch 17/50
 - 1s - loss: 0.7488 - val_loss: 0.7392
Epoch 18/50
 - 1s - loss: 0.7472 - val_loss: 0.7376
Epoch 19/50
 - 1s - loss: 0.7454 - val_loss: 0.7358
Epoch 20/50
 - 1s - loss: 0.7435 - val_loss: 0.7339
Epoch 21/50
 - 1s - loss: 0.7414 - val_loss: 0.7316
Epoch 22/50
 - 1s - loss: 0.7390 - val_loss: 0.7292
Epoch 23/50
 - 1s - loss: 0.7362 - val_loss: 0.7261
Epoch 24/50
 - 1s - loss: 0.7330 - val_loss: 0.7230
Epoch 25/50
 - 1s - loss: 0.7297 - val_loss: 0.7195
Epoch 26/50
 - 1s - loss: 0.7262 - val_loss: 0.7160
Epoch 27/50
 - 1s - loss: 0.7225 - val_loss: 0.7120
Epoch 28/50
 - 1s - loss: 0.7183 - val_loss: 0.7077
Epoch 29/50
 - 1s - loss: 0.7144 - val_loss: 0.7033
Epoch 30/50
 - 1s - loss: 0.7102 - val_loss: 0.6994
Epoch 31/50
 - 1s - loss: 0.7070 - val_loss: 0.6956
Epoch 32/50


Epoch 19/50
 - 1s - loss: 0.7461 - val_loss: 0.7366
Epoch 20/50
 - 1s - loss: 0.7443 - val_loss: 0.7348
Epoch 21/50
 - 1s - loss: 0.7424 - val_loss: 0.7329
Epoch 22/50
 - 1s - loss: 0.7403 - val_loss: 0.7310
Epoch 23/50
 - 1s - loss: 0.7379 - val_loss: 0.7283
Epoch 24/50
 - 1s - loss: 0.7352 - val_loss: 0.7256
Epoch 25/50
 - 1s - loss: 0.7324 - val_loss: 0.7228
Epoch 26/50
 - 1s - loss: 0.7291 - val_loss: 0.7195
Epoch 27/50
 - 1s - loss: 0.7257 - val_loss: 0.7157
Epoch 28/50
 - 1s - loss: 0.7217 - val_loss: 0.7120
Epoch 29/50
 - 1s - loss: 0.7182 - val_loss: 0.7075
Epoch 30/50
 - 1s - loss: 0.7143 - val_loss: 0.7038
Epoch 31/50
 - 1s - loss: 0.7111 - val_loss: 0.7004
Epoch 32/50
 - 1s - loss: 0.7080 - val_loss: 0.6967
Epoch 33/50
 - 1s - loss: 0.7050 - val_loss: 0.6939
Epoch 34/50
 - 1s - loss: 0.7023 - val_loss: 0.6907
Epoch 35/50
 - 1s - loss: 0.6998 - val_loss: 0.6879
Epoch 36/50
 - 1s - loss: 0.6974 - val_loss: 0.6856
Epoch 37/50
 - 1s - loss: 0.6957 - val_loss: 0.6838
Epoch 38/50


Epoch 25/50
 - 1s - loss: 0.7355 - val_loss: 0.7266
Epoch 26/50
 - 1s - loss: 0.7328 - val_loss: 0.7238
Epoch 27/50
 - 1s - loss: 0.7297 - val_loss: 0.7205
Epoch 28/50
 - 1s - loss: 0.7265 - val_loss: 0.7170
Epoch 29/50
 - 1s - loss: 0.7230 - val_loss: 0.7134
Epoch 30/50
 - 1s - loss: 0.7194 - val_loss: 0.7096
Epoch 31/50
 - 1s - loss: 0.7161 - val_loss: 0.7059
Epoch 32/50
 - 1s - loss: 0.7133 - val_loss: 0.7022
Epoch 33/50
 - 1s - loss: 0.7096 - val_loss: 0.6990
Epoch 34/50
 - 1s - loss: 0.7065 - val_loss: 0.6953
Epoch 35/50
 - 1s - loss: 0.7033 - val_loss: 0.6921
Epoch 36/50
 - 1s - loss: 0.7003 - val_loss: 0.6891
Epoch 37/50
 - 1s - loss: 0.6978 - val_loss: 0.6859
Epoch 38/50
 - 1s - loss: 0.6954 - val_loss: 0.6836
Epoch 39/50
 - 1s - loss: 0.6936 - val_loss: 0.6816
Epoch 40/50
 - 1s - loss: 0.6923 - val_loss: 0.6801
Epoch 41/50
 - 1s - loss: 0.6909 - val_loss: 0.6787
Epoch 42/50
 - 1s - loss: 0.6897 - val_loss: 0.6771
Epoch 43/50
 - 1s - loss: 0.6882 - val_loss: 0.6752
Epoch 44/50


In [29]:
test_X

array([[ 0.31212986, -0.6       , -0.77546525, -0.05263162,  0.20000005,
        -0.80930036],
       [-0.9766829 , -0.6       , -0.77546525, -0.05263162, -0.6       ,
        -0.904942  ],
       [-0.5782522 , -0.6       , -0.77546525, -0.05263162, -0.19999999,
        -0.84110457],
       ...,
       [-0.92102396, -0.19999999, -0.9638148 , -0.31578946, -0.19999999,
        -0.94060534],
       [-0.9262204 , -0.19999999, -0.9638148 , -0.31578946,  0.20000005,
        -0.87223226],
       [-0.9840721 , -0.19999999, -0.9638148 , -0.31578946,  0.20000005,
        -0.92875123]], dtype=float32)

In [28]:
yhat

array([[ 0.5187012 ],
       [-1.0073264 ],
       [-0.88969034],
       ...,
       [-0.9344703 ],
       [ 0.53998315],
       [ 0.53776383]], dtype=float32)

In [19]:
# # make a prediction
yhat = model.predict(test_X)

test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# # invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -6:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# # invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -6:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
count = 0

for i in range(0,len(inv_yhat)):
    if int(inv_yhat[i]) > 99:
        inv_yhat[i] = 1
    else: inv_yhat[i] = 0
    if int(inv_y[i]) > 99:
                inv_y[i] = 1
    else: inv_y[i] = 0
    if inv_y[i] == inv_yhat[i]:count +=1

    accuracy = (count/len(yhat))*100
    runs[j] = accuracy
    print("Accuracy:"+ str(accuracy))

for i in range(0,len(runs)):
    print("Accuracy:" + str(runs[i]))
mean = sum(runs)/len(runs)
print("n_epoch: {}".format(n_epoch))
print("Mean_Accuracy: {}".format(mean))
print("Standard_Deviation: {}".format(np.std(runs, 0)))

Accuracy:0.0
Accuracy:0.017208742040956806
Accuracy:0.03441748408191361
Accuracy:0.03441748408191361
Accuracy:0.03441748408191361
Accuracy:0.05162622612287042
Accuracy:0.06883496816382723
Accuracy:0.08604371020478403
Accuracy:0.08604371020478403
Accuracy:0.10325245224574084
Accuracy:0.10325245224574084
Accuracy:0.12046119428669765
Accuracy:0.13766993632765445
Accuracy:0.15487867836861124
Accuracy:0.15487867836861124
Accuracy:0.17208742040956806
Accuracy:0.18929616245052486
Accuracy:0.20650490449148168
Accuracy:0.20650490449148168
Accuracy:0.20650490449148168
Accuracy:0.20650490449148168
Accuracy:0.20650490449148168
Accuracy:0.22371364653243847
Accuracy:0.2409223885733953
Accuracy:0.2409223885733953
Accuracy:0.2409223885733953
Accuracy:0.2581311306143521
Accuracy:0.2753398726553089
Accuracy:0.2925486146962657
Accuracy:0.3097573567372225
Accuracy:0.3097573567372225
Accuracy:0.32696609877817934
Accuracy:0.34417484081913613
Accuracy:0.34417484081913613
Accuracy:0.3613835828600929
Accuracy:

Accuracy:17.89709172259508
Accuracy:17.89709172259508
Accuracy:17.914300464636035
Accuracy:17.931509206676992
Accuracy:17.931509206676992
Accuracy:17.94871794871795
Accuracy:17.965926690758906
Accuracy:17.983135432799862
Accuracy:18.00034417484082
Accuracy:18.00034417484082
Accuracy:18.017552916881776
Accuracy:18.034761658922733
Accuracy:18.034761658922733
Accuracy:18.034761658922733
Accuracy:18.034761658922733
Accuracy:18.05197040096369
Accuracy:18.05197040096369
Accuracy:18.069179143004646
Accuracy:18.086387885045603
Accuracy:18.10359662708656
Accuracy:18.10359662708656
Accuracy:18.120805369127517
Accuracy:18.138014111168474
Accuracy:18.15522285320943
Accuracy:18.172431595250387
Accuracy:18.189640337291344
Accuracy:18.189640337291344
Accuracy:18.2068490793323
Accuracy:18.2068490793323
Accuracy:18.224057821373258
Accuracy:18.241266563414214
Accuracy:18.25847530545517
Accuracy:18.275684047496128
Accuracy:18.292892789537085
Accuracy:18.31010153157804
Accuracy:18.327310273619
Accuracy:18

Accuracy:29.70228876269145
Accuracy:29.70228876269145
Accuracy:29.719497504732406
Accuracy:29.719497504732406
Accuracy:29.73670624677336
Accuracy:29.73670624677336
Accuracy:29.73670624677336
Accuracy:29.73670624677336
Accuracy:29.753914988814316
Accuracy:29.771123730855276
Accuracy:29.788332472896233
Accuracy:29.788332472896233
Accuracy:29.805541214937186
Accuracy:29.822749956978146
Accuracy:29.839958699019103
Accuracy:29.857167441060056
Accuracy:29.857167441060056
Accuracy:29.857167441060056
Accuracy:29.857167441060056
Accuracy:29.874376183101013
Accuracy:29.891584925141974
Accuracy:29.90879366718293
Accuracy:29.926002409223884
Accuracy:29.943211151264844
Accuracy:29.943211151264844
Accuracy:29.9604198933058
Accuracy:29.977628635346754
Accuracy:29.99483737738771
Accuracy:30.01204611942867
Accuracy:30.01204611942867
Accuracy:30.029254861469628
Accuracy:30.029254861469628
Accuracy:30.029254861469628
Accuracy:30.04646360351058
Accuracy:30.04646360351058
Accuracy:30.04646360351058
Accurac

Accuracy:43.779039752194116
Accuracy:43.79624849423507
Accuracy:43.81345723627603
Accuracy:43.830665978316986
Accuracy:43.84787472035794
Accuracy:43.86508346239889
Accuracy:43.88229220443986
Accuracy:43.89950094648081
Accuracy:43.91670968852177
Accuracy:43.93391843056273
Accuracy:43.951127172603684
Accuracy:43.96833591464464
Accuracy:43.9855446566856
Accuracy:44.002753398726554
Accuracy:44.01996214076751
Accuracy:44.03717088280847
Accuracy:44.054379624849425
Accuracy:44.07158836689038
Accuracy:44.07158836689038
Accuracy:44.07158836689038
Accuracy:44.07158836689038
Accuracy:44.08879710893134
Accuracy:44.106005850972295
Accuracy:44.106005850972295
Accuracy:44.106005850972295
Accuracy:44.106005850972295
Accuracy:44.106005850972295
Accuracy:44.106005850972295
Accuracy:44.12321459301325
Accuracy:44.14042333505421
Accuracy:44.14042333505421
Accuracy:44.157632077095165
Accuracy:44.17484081913612
Accuracy:44.17484081913612
Accuracy:44.19204956117708
Accuracy:44.19204956117708
Accuracy:44.20925

Accuracy:59.198072620891416
Accuracy:59.21528136293237
Accuracy:59.23249010497332
Accuracy:59.24969884701429
Accuracy:59.266907589055236
Accuracy:59.28411633109619
Accuracy:59.28411633109619
Accuracy:59.28411633109619
Accuracy:59.28411633109619
Accuracy:59.30132507313716
Accuracy:59.30132507313716
Accuracy:59.30132507313716
Accuracy:59.31853381517811
Accuracy:59.33574255721906
Accuracy:59.35295129926003
Accuracy:59.37016004130098
Accuracy:59.38736878334194
Accuracy:59.4045775253829
Accuracy:59.4045775253829
Accuracy:59.42178626742385
Accuracy:59.43899500946481
Accuracy:59.43899500946481
Accuracy:59.43899500946481
Accuracy:59.43899500946481
Accuracy:59.45620375150577
Accuracy:59.45620375150577
Accuracy:59.47341249354672
Accuracy:59.47341249354672
Accuracy:59.47341249354672
Accuracy:59.49062123558768
Accuracy:59.50782997762863
Accuracy:59.52503871966959
Accuracy:59.52503871966959
Accuracy:59.52503871966959
Accuracy:59.52503871966959
Accuracy:59.54224746171055
Accuracy:59.54224746171055
A